In [501]:
d = 12
p = next_prime(2^d)
n = d
k = ceil(sqrt(d) + log(d, 2))
s = randint(1, p - 1)

print(f"p = {p}")
print(f"n = {n}")
print(f"k = {k}")
print(f"s = {s}")

p = 4099
n = 12
k = 8
s = 424


In [502]:
def represent_in_Zq(val, max):
    return val % max

def msb_k(y, k, p):
    B = p // (2 ** k)
    return y // B

In [503]:
def generate_pairs(N, q, k, s):
    pairs = []
    for i in range(N):
        x_i = randint(0, q - 1)
        u_i = msb_k((s * x_i) % q, k, q)
        pairs.append((x_i, u_i))
    return pairs

pairs = generate_pairs(n_ex, p_ex, k_ex, s_ex)
print(pairs)

[(2745, 87), (2671, 74), (807, 42), (729, 41), (2339, 138), (2018, 39), (2943, 206), (3608, 202), (2115, 105), (1592, 156), (1242, 23), (2141, 20)]


In [504]:
def build_integer_lattice(oracle_inputs, secon, p, k):
    l = len(oracle_inputs) + 2  # Dimension of the lattice
    basis_vectors = []

    # Add basis vectors for oracle inputs
    for i in range(l-2):
        p_vector = [0] * (l)
        p_vector[i] = p
        basis_vectors.append(p_vector)


    # Add the last basis vector with appropriate values
    A = 1 / (2 ** k)
    M = p * (2 ** k)
    last_basis_vector = oracle_inputs + [A,0]
    basis_vectors.append(last_basis_vector)
    last_last = secon + [0, M]
    basis_vectors.append(last_last)
    return Matrix(QQ, basis_vectors)

B = p / (2 ** k)
input_lattice = build_integer_lattice([x_i for x_i, _ in pairs_ex], [-B * u_i for _, u_i in pairs_ex], p_ex,k_ex)
print(input_lattice)

[       4099           0           0           0           0           0           0           0           0           0           0           0           0           0]
[          0        4099           0           0           0           0           0           0           0           0           0           0           0           0]
[          0           0        4099           0           0           0           0           0           0           0           0           0           0           0]
[          0           0           0        4099           0           0           0           0           0           0           0           0           0           0]
[          0           0           0           0        4099           0           0           0           0           0           0           0           0           0]
[          0           0           0           0           0        4099           0           0           0           0           0           0      

In [505]:
def reduce_lattice(lattice):
    return lattice.LLL()

reduced_lattice = reduce_lattice(input_lattice)
print(reduced_lattice)

[        0         0         0         0         0         0         0         0         0         0         0         0 -4099/256         0]
[     -348       787     -1021       444      -183      -230       665       436       -13       254       414       487    55/128         0]
[     -746      -598      -469      -603      -887     -1294       177      -149      -287       -68      -950        31   -47/128         0]
[     -930      -229      -255     -1499       147      -756      -131      -955       884      -876       541      -324   359/128         0]
[    -1129      1128        21        27      -205     -1288      -375       802       747     -1037      -141      -552     77/32         0]
[      308      -885      -180      -817       586     -1257       872      -433       624       105       623      1218 -1181/256         0]
[     1048       928      -506       -65      -721      -391      -919      1561      -432      -388      -116       418   187/256         0]
[     

In [506]:
def get_secret_from_reduced_lattice(reduced_lattice, q, k):
    shortest_vector = reduced_lattice.row(-1)
    s = (shortest_vector[-2] * (2 ** k)).ceil()
    return represent_in_Zq(s, q)

found_secret = get_secret_from_reduced_lattice(reduced_lattice, p, k)
print(found_secret)

1997


In [507]:
def verify_secret(ss, x_values, u_values, q, k):
    for x, u in zip(x_values, u_values):
        if msb_k((ss * x) % q, k, q) != u:
            return False
    return True

# Verificar o segredo extraído
t = [x for x, _ in pairs]
u = [u for _, u in pairs] 
is_correct = verify_secret(found_secret, t, u, p, k)

if is_correct:
    print("O segredo extraído está correto.")
else:
    print("O segredo extraído está incorreto.")

O segredo extraído está correto.


In [509]:
def test_for_d(dd):
    pp = next_prime(2**dd)
    nn = dd
    kk = ceil(sqrt(dd) + log(dd, 2))
    ss = randint(1, pp - 1)

    
    ppairs = generate_pairs(nn, pp, kk, ss)

    BB = pp / (2 ** kk)
    iinput_lattice = build_integer_lattice([xx for xx, _ in ppairs], [-BB * uu for _, uu in ppairs], pp, kk)
    
    rreduced_lattice = reduce_lattice(iinput_lattice)
    
    ffound_secret = get_secret_from_reduced_lattice(rreduced_lattice, pp, kk)

    
    tt = [x for x, _ in ppairs]
    uu = [u for _, u in ppairs] 
    iis_correct = verify_secret(ffound_secret, tt, uu, pp, kk)
    
    return iis_correct

for i in range(10, 20):
    print(f"Testando para d = {i}...")
    print(test_for_d(i))

Testando para d = 10...
True
Testando para d = 11...
True
Testando para d = 12...
True
Testando para d = 13...
True
Testando para d = 14...
True
Testando para d = 15...
True
Testando para d = 16...
True
Testando para d = 17...
True
Testando para d = 18...
True
Testando para d = 19...
True
